In [1]:
import importlib
from pprint import pprint

import pandas as pd
from tqdm.auto import tqdm

from cuery import search
from cuery.seo import geo
from cuery.utils import set_env

set_env(apify_secrets=False)

# LLM prompting with web search

In [2]:
# model = "openai/gpt-4.1-mini"
model = "google/gemini-2.5-flash"
# model = "xai/grok-3"
# model = "deepseek/deepseek-chat"

prompts = [
    "Mejores SUVs en España 2025 precio-calidad",
    "Cuáles son los mejores SUVs chinos dispnibles en España 2025",
]

In [7]:
importlib.reload(search)

pbar = tqdm(total=len(prompts))

policies = {
    "timeout": 30,
    "n_concurrent": 10,
    "retries": 3,
    "wait_max": 10,
    "fallback": search.SearchResult.fallback(),
    "pbar": pbar,
    "min_iters": 1,
    "timer": True,
}

responses = await search.gather(prompts, model=model, policies=policies, country="es")
responses.to_pandas()

  0%|          | 0/2 [00:00<?, ?it/s]

2025-09-22 12:29:33 [cuery|INFO] q(gemini-2.5-flash): t=11.00s
2025-09-22 12:29:38 [cuery|INFO] q(gemini-2.5-flash): t=15.65s


prompt  \
0         Mejores SUVs en España 2025 precio-calidad   
1  Cuáles son los mejores SUVs chinos dispnibles ...   

                                              answer  \
0  A la hora de buscar los mejores SUVs en España...   
1  Los SUVs chinos han ganado una presencia signi...   

                                             sources  
0  [{'title': 'autohero.com', 'url': 'https://www...  
1  [{'title': 'carwow.es', 'url': 'https://www.ca...

# Competitor extraction

In [4]:
importlib.reload(geo)

competitors = await geo.find_competitors(
    brands=["BYD"],
    sector="automotive",
    market="Spain",
    max_count=5,
)
competitors

2025-09-18 11:35:30 [cuery|INFO] Searching for competitor brands with prompt:
Which are the 5 most important competing brands for ['BYD'] in the 'automotive' sector in the 'Spain' market
2025-09-18 11:35:37 [cuery|INFO] Extracting competitor names from search result
2025-09-18 11:35:38 [cuery|INFO] Found these new competitors brands: ['kia', 'renault', 'mg', 'toyota', 'tesla']


['kia', 'renault', 'mg', 'toyota', 'tesla']

In [ ]:
competitors = ["toyota", "kia", "mg", "renault", "omoda"]

In [5]:
from cuery.seo import geo

importlib.reload(geo)

prompts = await geo.generate_prompts(
    n=10,
    language="en",
    sector=None,  # "automotive",
    market=None,  # "Spain",
    brands=["BYD"] + competitors,
    include_brands="never",
)

pprint(prompts)

2025-09-18 11:35:40 [cuery|INFO] Generating search queries with prompt:
"Generate 10 unique, concise LLM prompts with one or more of the following intents: ['commercial', 'transactional']. Cover realistic user intentions like comparisons, alternatives, trust/regulatory, location specific queries etc. The prompts should be similar to Google search queries but adapted to how users would ask an LLM. Generate prompts in the 'en' language. If brand context helps, consider these brands: ['BYD', 'kia', 'renault', 'mg', 'toyota', 'tesla']. Do not mention any brand names explicitly in the queries though. Strictly return a JSON array of strings. No numbering, no prose, no code fences."
2025-09-18 11:35:45 [cuery|INFO] Generated 10 commercial search queries: [
  "Compare the latest electric SUVs available for lease in my region",
  "What are the safest hybrid cars to buy this year with good resale value?",
  "Show alternatives to compact electric vehicles under $35,000",
  "Which family SUVs qual

['Compare the latest electric SUVs available for lease in my region',
 'What are the safest hybrid cars to buy this year with good resale value?',
 'Show alternatives to compact electric vehicles under $35,000',
 'Which family SUVs qualify for government incentives in the US?',
 'Find electric sedans with fast-charging and long warranty coverage',
 'Are there reliable dealerships near me offering trade-ins for plug-in '
 'hybrids?',
 'List the best-reviewed mid-size EVs for long-distance commuting',
 'What plug-in hybrid SUVs have the lowest maintenance costs?',
 'Help me choose between two-seater electric sports cars for city driving',
 'Show trusted sources for certified used EVs with battery guarantees']


In [ ]:
importlib.reload(search)
importlib.reload(geo)

use_search = True
models = ["openai/gpt-4.1", "google/gemini-2.5-flash"]
# models = ["deepseek/deepseek-chat"]

df = await geo.query_ais(prompts, models, use_search=use_search)
df

  0%|          | 0/20 [00:00<?, ?it/s]

prompt  \
0  Are there reliable dealerships near me offerin...   
1  Compare the latest electric SUVs available for...   
2  Find electric sedans with fast-charging and lo...   
3  Help me choose between two-seater electric spo...   
4  List the best-reviewed mid-size EVs for long-d...   
5  Show alternatives to compact electric vehicles...   
6  Show trusted sources for certified used EVs wi...   
7  What are the safest hybrid cars to buy this ye...   
8  What plug-in hybrid SUVs have the lowest maint...   
9  Which family SUVs qualify for government incen...   

                      answer_google/gemini-2.5-flash  \
0  To provide you with reliable dealerships offer...   
1  Please tell me your region so I can accurately...   
2  Electric sedans offering both fast-charging ca...   
3  When choosing a two-seater electric sports car...   
4  For electric vehicle (EV) owners prioritizing ...   
5  For car buyers seeking electric vehicles under...   
6  When considering a certified used electric veh...   
7  To identify the safest hybrid cars with good r...   
8  Plug-in hybrid SUVs (PHEVs) generally offer lo...   
9  The US government offers incentives for the pu...   

                               answer_openai/gpt-4.1  \
0  Could you please let me know your current loca...   
1  Here’s a detailed comparison of the **latest e...   
2  Here are several notable electric sedans avail...   
3  Here are some compelling two-seater electric s...   
4  Here are some of the **best-reviewed mid‑size ...   
5  Here are several compelling **compact electric...   
6  Here are several **trusted sources** in the Un...   
7  Here’s a refined, well-structured overview (ov...   
8  Here’s a detailed, up-to-date look (as of Sept...   
9  Here is the current landscape (as of September...   

                     sources_google/gemini-2.5-flash  \
0                                                 []   
1                                                 []   
2  [{'title': 'edmunds.com', 'url': 'https://vert...   
3  [{'title': 'carwow.co.uk', 'url': 'https://www...   
4  [{'title': 'mbofcaldwell.com', 'url': 'https:/...   
5  [{'title': 'truecar.com', 'url': 'https://www....   
6  [{'title': 'bmwusa.com', 'url': 'https://www.b...   
7  [{'title': 'iseecars.com', 'url': 'https://www...   
8  [{'title': 'torquenews.com', 'url': 'https://w...   
9  [{'title': 'nerdwallet.com', 'url': 'https://w...   

                              sources_openai/gpt-4.1  
0                                                 []  
1  [{'title': 'Want to Lease an EV? The Tax Credi...  
2  [{'title': 'Hyundai Ioniq 6', 'url': 'https://...  
3  [{'title': 'MG Cyberster', 'url': 'https://en....  
4  [{'title': 'I drove the Lucid Air Touring for ...  
5  [{'title': 'Top 5 Best Electric Vehicles (EVs)...  
6  [{'title': 'EV Certified Used Vehicle Inspecti...  
7  [{'title': 'The Safest Hybrids of 2024-2025 - ...  
8  [{'title': '10 Plug-in Hybrids With The Lowest...  
9  [{'title': 'Electric Vehicles & Plug-In Hybrid...

In [ ]:
df.to_parquet("geo_results.parquet", index=False)

In [ ]:
df = df[[col for col in df if not any(x in col for x in ("_ranks", "_pos"))]].copy()
df

In [ ]:
importlib.reload(geo)

df2 = geo.add_brand_ranks(
    df,
    brands=["Opel", "Dacia", "BYD", "Kia", "Renault", "MG", "Omoda", "Jaecoo"],
)
df2

# All-in-one

In [6]:
importlib.reload(geo)

<module 'cuery.seo.geo' from '/Users/thomas/code/cuery/src/cuery/seo/geo.py'>

In [ ]:
rs = await geo.query_ais(
    prompts=["What is the ISS?", "What are the benifits of intermittent fasting?"],
    models=["openai/gpt-4.1-mini", "google/ai-overview"],
    use_search=True,
    to_pandas=True,
)

rs

  0%|          | 0/4 [00:00<?, ?it/s]

prompt  \
0  What are the benifits of intermittent fasting?   
1                                What is the ISS?   

                           answer_google/ai-overview  \
0  Intermittent fasting (IF) is an eating pattern...   
1  The ISS is the International Space Station, a ...   

                          answer_openai/gpt-4.1-mini  \
0  Intermittent fasting (IF) involves alternating...   
1  The International Space Station (ISS) is a lar...   

                          sources_google/ai-overview  \
0  [{'title': 'Intermittent Fasting: What is it, ...   
1  [{'title': 'International Space Station - NASA...   

                         sources_openai/gpt-4.1-mini  
0  [{'title': 'Intermittent fasting for weight lo...  
1  [{'title': 'International Space Station - NASA...

In [2]:
importlib.reload(search)
importlib.reload(geo)

params = {
    "brands": ["Peugeot", "Opel"],
    "models": ["openai/gpt-4.1-mini", "google/ai-overview", "google/gemini-2.5-flash"],
    "prompts": [
        "What are the benefits of an SUV?",
        "What are the pros and cons of electric vehicles?",
    ],
    # "prompts": ["What is the ISS?", "What are the benifits of intermittent fasting?"],
    "prompts_max": 10,
    "prompt_intents": ["commercial", "transactional"],
    "prompt_language": "English",
    "brands_in_prompt": "never",
    "competitors": ["BYD", "MG", "Renault", "Kia"],
    "competitors_max": 10,
    "sector": "automotive, SUVs",
    "market": "Unites States",
    "use_search": True,
    "search_country": "US",
}

cfg = geo.GeoConfig(**params)

In [3]:
result = await geo.analyse(cfg)
result

2025-09-22 12:44:50 [cuery|INFO] Querying LLMs with

GeoConfig
—————————
{
  "brands": [
    "Peugeot",
    "Opel"
  ],
  "models": [
    "openai/gpt-4.1-mini",
    "google/ai-overview",
    "google/gemini-2.5-flash"
  ],
  "prompts": [
    "What are the benefits of an SUV?",
    "What are the pros and cons of electric vehicles?"
  ],
  "prompts_max": 10,
  "prompt_intents": [
    "commercial",
    "transactional"
  ],
  "prompt_language": "English",
  "brands_in_prompt": "never",
  "competitors": [
    "BYD",
    "MG",
    "Renault",
    "Kia"
  ],
  "competitors_max": 10,
  "competitors_model": "openai/gpt-4.1",
  "sector": "automotive, SUVs",
  "market": "Unites States",
  "use_search": true,
  "search_country": "US"
}

2025-09-22 12:44:50 [cuery|INFO] Using these seed brands: ['opel', 'peugeot'], and these competitors: ['mg', 'byd', 'renault', 'kia']
2025-09-22 12:44:50 [cuery|INFO] Searching for competitor brands with prompt:
Which are the 6 most important competing brands for ['o

  0%|          | 0/30 [00:00<?, ?it/s]

2025-09-22 12:45:07 [cuery|WARNING] Need second request to get AI overview for prompt: Locate dealerships near me that have new SUV models in stock with lease offers.
2025-09-22 12:45:07 [cuery|WARNING] Need second request to get AI overview for prompt: Compare hybrid SUVs with gasoline SUVs for cost of ownership in the US.
2025-09-22 12:45:08 [cuery|WARNING] Need second request to get AI overview for prompt: What are the benefits of an SUV?
2025-09-22 12:45:08 [cuery|INFO] q(hasdata-aio): t=3.52s
2025-09-22 12:45:09 [cuery|WARNING] Need second request to get AI overview for prompt: Show me the best midsize SUVs for families available in the United States right now.
2025-09-22 12:45:09 [cuery|WARNING] Need second request to get AI overview for prompt: Which SUVs offer the highest safety ratings for city driving in America?
2025-09-22 12:45:10 [cuery|WARNING] Need second request to get AI overview for prompt: What are the top-rated alternatives to luxury SUVs under $50,000 in the US mar

prompt  \
0  Are there any SUVs with recent recalls or regu...   
1  Compare hybrid SUVs with gasoline SUVs for cos...   
2  Find SUVs with the best fuel efficiency that a...   
3  Locate dealerships near me that have new SUV m...   
4  Show me the best midsize SUVs for families ava...   
5                   What are the benefits of an SUV?   
6  What are the most reliable used SUV models for...   
7   What are the pros and cons of electric vehicles?   
8  What are the top-rated alternatives to luxury ...   
9  Which SUVs offer the highest safety ratings fo...   

   brand_mentioned_in_answer_count  brand_mentioned_in_sources_count  \
0                                0                                 0   
1                                0                                 0   
2                                0                                 0   
3                                0                                 0   
4                                0                                 0   
5                                0                                 0   
6                                0                                 0   
7                                0                                 0   
8                                0                                 0   
9                                0                                 0   

   competitor_mentioned_in_answer_count  \
0                                     2   
1                                     2   
2                                     3   
3                                     2   
4                                     3   
5                                     0   
6                                     3   
7                                     0   
8                                     2   
9                                     3   

   competitor_mentioned_in_sources_count  \
0                                      1   
1                                      2   
2                                      1   
3                                      2   
4                                      0   
5                                      0   
6                                      0   
7                                      0   
8                                      0   
9                                      1   

   brand_mentioned_in_answer_gpt_4_1_mini  \
0                                   False   
1                                   False   
2                                   False   
3                                   False   
4                                   False   
5                                   False   
6                                   False   
7                                   False   
8                                   False   
9                                   False   

  brand_position_in_answer_gpt_4_1_mini  \
0                                  None   
1                                  None   
2                                  None   
3                                  None   
4                                  None   
5                                  None   
6                                  None   
7                                  None   
8                                  None   
9                                  None   

   brand_mentioned_in_sources_gpt_4_1_mini  \
0                                    False   
1                                    False   
2                                    False   
3                                    False   
4                                    False   
5                                    False   
6                                    False   
7                                    False   
8                                    False   
9                                    False   

  brand_position_in_sources_gpt_4_1_mini  \
0                                   None   
1                                   None   
2                                   None   
3                       

In [15]:
result.to_parquet("geo_results.parquet", index=False)

In [16]:
result = pd.read_parquet("geo_results.parquet")
result

prompt  own_in_txt_count  \
0  Are there any trade-in deals for upgrading to ...                 0   
1  Compare hybrid compact cars available in Spain...                 1   
2  Find certified used cars with warranty near Ba...                 0   
3  Find safe petrol cars for long-distance travel...                 1   
4  Show me 7-seater cars with the best resale val...                 1   
5  Show me affordable hatchback lease options in ...                 2   
6  What are the financing options for new sedans ...                 2   
7  What government incentives exist for buying ec...                 0   
8  Which vehicles have the lowest insurance costs...                 0   
9            best electric SUVs for families in 2024                 0   

   own_in_refs_count  cmp_in_txt_count  cmp_in_refs_count own_in_txt_gpt_4_1  \
0                  0                 0                  0                  ❌   
1                  0                 3                  2                  ✅   
2                  0                 3                  2                  ❌   
3                  0                 3                  0                  ❌   
4                  0                 3                  0                  ❌   
5                  0                 3                  1                  ✅   
6                  1                 3                  1                  ❌   
7                  0                 1                  1                  ❌   
8                  0                 2                  0                  ❌   
9                  0                 3                  2                  ❌   

   own_in_txt_pos_gpt_4_1 own_in_refs_gpt_4_1 own_in_refs_pos_gpt_4_1  \
0                     NaN                   ❌                    None   
1                     3.0                   ❌                    None   
2                     NaN                   ❌                    None   
3                     NaN                   ❌                    None   
4                     NaN                   ❌                    None   
5                     4.0                   ❌                    None   
6                     NaN                   ❌                    None   
7                     NaN                   ❌                    None   
8                     NaN                   ❌                    None   
9                     NaN                   ❌                    None   

  own_in_txt_gemini_2_5_flash  ...  \
0                           ❌  ...   
1                           ❌  ...   
2                           ❌  ...   
3                           ❌  ...   
4                           ✅  ...   
5                           ✅  ...   
6                           ✅  ...   
7                           ❌  ...   
8                           ❌  ...   
9                           ❌  ...   

                                        text_gpt_4_1 references_deepseek_chat  \
0  Here’s an overview of **current trade-in and u...                       []   
1  Below is a detailed comparison of popular hybr...                       []   
2  Here are several dealerships and programs offe...                       []   
3  Here are some excellent petrol-powered cars su...                       []   
4  Here are some of the **7‑seater vehicles** ava...                       []   
5  Here’s a structured overview of affordable hat...                       []   
6  Here is an in-depth overview of financing opti...                       []   
7  Here’s a comprehensive overview of current gov...                       []   
8  Here’s what you need to know about the vehicle...                       []   
9  Here are some of the **best electric SUVs for ...                       []   

                         references_gemini_2_5_flash  \
0  [{'title': 'apple.com', 'url': 'https://www.ap...   
1  [{'title': 'autogiz.com', 'url': 'https://www....   
2  [{'title': 'tresdarc.com', 'url': 'https://tre...   
3  [{'title

In [5]:
importlib.reload(geo)

own = ["Peugeot"]
competitors = [
    "citroën",
    "mercedes-benz",
    "volkswagen",
    "mg",
    "dacia",
    "renault",
    "toyota",
    "hyundai",
    "seat",
    "kia",
]

geo.summarize_ranks(result, models=cfg.models, own=own, competitors=competitors, emoji_flags=True)

NameError: name 'cfg' is not defined

In [1]:
df = result
df

NameError: name 'result' is not defined

In [41]:
importlib.reload(geo)

result = result[[col for col in result if not any(x in col for x in ("_ranks", "_pos"))]].copy()
result = geo.add_brand_ranks(
    result,
    brands=cfg.brands_seed
    + [
        "citroën",
        "mercedes-benz",
        "volkswagen",
        "mg",
        "dacia",
        "renault",
        "toyota",
        "hyundai",
        "seat",
        "kia",
    ],
)
result

prompt  \
0        approved used cars warranty Spain   
1  best electric SUVs for families in 2024   
2              best suv offers Madrid 2024   
3                 car safety ratings Spain   
4          compact crossover reviews Spain   
5      compare hybrid vs diesel cars Spain   
6   eco-friendly city cars available Spain   
7        electric car tax incentives Spain   
8      family hatchback alternatives Spain   
9           lease vs finance new car Spain   

                         text_deepseek/deepseek-chat  \
0  I understand you're looking for information ab...   
1  Based on my research, here are the best electr...   
2  I understand you're looking for the best SUV o...   
3  I understand you're looking for car safety rat...   
4  I understand you're looking for compact crosso...   
5  ## Hybrid vs Diesel Cars in Spain: Comparison\...   
6  Based on your query about eco-friendly city ca...   
7  ## Electric Car Tax Incentives in Spain\n\n###...   
8  Based on your query about family hatchback alt...   
9  When deciding between leasing and financing a ...   

                        text_google/gemini-2.5-flash  \
0  In Spain, the warranty for approved used cars ...   
1  For families seeking an eco-friendly and pract...   
2  While specific "best" offers can be subjective...   
3  Car safety ratings in Spain are primarily dete...   
4  In the Spanish market, compact crossovers cont...   
5  ## Hybrid vs. Diesel Cars in Spain: A Comprehe...   
6  Spain offers a growing selection of eco-friend...   
7  Spain offers several tax incentives and subsid...   
8  For families in Spain looking for alternatives...   
9  When considering a new car in Spain, both leas...   

                                 text_openai/gpt-4.1  \
0  If you're looking into warranties for "approve...   
1  Here’s a well-rounded overview of the best ele...   
2  Aquí tienes algunas de las **mejores ofertas d...   
3  Aquí tienes una visión clara y actualizada sob...   
4  Aquí tienes una visión actualizada (septiembre...   
5  Here’s a thorough comparison of hybrid versus ...   
6  Here’s a comprehensive overview of eco‑friendl...   
7  Here is an up-to-date overview (as of Septembe...   
8  If you're looking for practical alternatives t...   
9  Aquí tienes una comparativa clara y detallada ...   

  references_deepseek/deepseek-chat  \
0                                []   
1                                []   
2                                []   
3                                []   
4                                []   
5                                []   
6                                []   
7                                []   
8                                []   
9                                []   

                  references_google/gemini-2.5-flash  \
0  [{'title': 'mylawyerinspain.com', 'url': 'http...   
1  [{'title': 'kbb.com', 'url': 'https://www.kbb....   
2  [{'title': 'coches.net', 'url': 'https://www.c...   
3  [{'title': 'byd.com', 'url': 'https://www.byd....   
4  [{'title': 'youtube.com', 'url': 'https://www....   
5  [{'title': 'globalpetrolprices.com', 'url': 'h...   
6  [{'title': 'euroweeklynews.com', 'url': 'https...   
7  [{'title': 'electrive.com', 'url': 'https://ww...   
8  [{'title': '6wresearch.com', 'url': 'https://v...   
9  [{'title': 'autoleaserenting.es', 'url': 'http...   

                           references_openai/gpt-4.1  \
0  [{'title': '! Spanish News Today - Extended Wa...   
1  [{'title': 'Edmunds Top Rated Awards for 2024 ...   
2  [{'title': 'BYD llega a Grupo Gamboa: descubre...   
3  [{'title': 'DGT - Clasificación de seguridad d...   
4  [{'title': 'Citroën C3 Aircross, Coche del Año...   
5  [{'title': 'Car sales in Spain by energy sourc...   
6  [{'title': 'Best Small Electric Car 2025: Pric...   
7  [{'title': 'Spain extends Moves III programme ...   
8  [{'title': 'Top 5 Station Wagons to Buy in 202...   
9  [{'title': 'Leasing vs Renting de Coches en Es...   

              

In [3]:
result

NameError: name 'result' is not defined